# **Censo Escolar** - Microdados da Educação Básica

Os dados do Censo Escolar foram baixados do site do INEP, de 2016 a 2023, em formato csv e são tratados neste script.

## Configurações

In [ ]:
import sys
import pandas as pd
import warnings
import os
from datetime import datetime

sys.path.append('/apoiodev/')
from geocoder import Geocoder 
from connect import conn, command_bcp
from utils import endereco_geocode
from config import cd_CensoEscolar, cd_Bcp

warnings.filterwarnings("ignore")

In [ ]:
ano_inicial = input("Digite o ano inicial: ")
ano_inicial = int(ano_inicial)
ano_final = input("Digite o ano final: ")
ano_final = int(ano_final)

anos = list(range(ano_inicial, ano_final + 1))

current_datetime = datetime.now()
inicio = current_datetime.strftime('%Y-%m-%d %H:%M:%S')

### Dicionários

In [ ]:
dicionario0 = pd.read_excel(f"{cd_CensoEscolar}dicionario2023.xlsx")
dicionario1 = dicionario0.dropna(subset='indicador')
dicionario1['atributo'] = dicionario1['atributo'].fillna('')
dicionario1['identificacao'] = dicionario1['indicador']
dicionario1.loc[dicionario1['atributo'] != '', 'identificacao'] = dicionario1['indicador'].astype(str) + "|" + dicionario1['atributo'].astype(str)
columns_dict = dicionario1.set_index('variavel')['identificacao'].to_dict()

points = pd.read_sql("SELECT * FROM cadastro_d_points;", conn)
points['endereco'] = points['endereco'].str.strip().str.lower()
points_dict = points.set_index('endereco')['idpoint'].to_dict()

### Funções

In [ ]:

def transform (base_inicial):
    ### base1 = base para alimentação da tabela dimensão
    base1 = base_inicial.copy()
    base1 = base1[list(columns_dict.keys())]
    base1 = base1.rename(columns=columns_dict)
    base1['endereco'] = endereco_geocode(base1, 'logradouro', 'numero', 'bairro', 'municipio', 'uf', 'cep').str.strip().str.lower()

    ### base2 = base para alimentação das tabelas fato
    base2 = base1.copy()
    base2['iddata'] = pd.to_datetime(base2['censo'].astype(str) + '-01-01')
    base2 = base2.drop(columns=['censo', 'uf', 'municipio', 'escola', 'idcategoria', 'logradouro', 'numero', 'bairro', 'cep', 'endereco'])
    id_vars = ['iddata', 'idinep_orig']
    base2 = base2.melt(id_vars=id_vars, var_name='variavel', value_name='valor')
    base2[['indicador', 'atributo']] = base2['variavel'].str.split("|", expand=True)
    del base2['variavel']

    base3 = base2.dropna(subset='valor').reset_index().copy()
    base3['valor'] = base3['valor'].astype(int)
    base3 = base2.loc[base2['valor'] > 0]

    return base1, base3


def gera_fato (df, indicador, atributo, colunas):
    df_fato = df.loc[df['indicador'] == indicador].reset_index(drop=True).copy()
    df_fato = df_fato.rename(columns={'atributo': atributo})
    df_fato['iddata'] = pd.to_datetime(df_fato['iddata'])
    df_fato[atributo] = df_fato[atributo].astype(str).str.replace('.0', '')
    df_fato[atributo] = df_fato[atributo].astype(int)
    df_fato['valor'] = df_fato['valor'].astype(int)
    df_fato['idinep_orig'] = df_fato['idinep_orig'].astype(int)
    df_fato = df_fato[colunas]
    return df_fato



def trata_dimensoes (df):

    escolas0 = df[['censo', 'idinep_orig', 'idcategoria', 'endereco', 'escola']].copy()
    escolas0 = escolas0.sort_values(by=['idinep_orig', 'censo'], ascending=False)

    ### escolas1  = seleciona último registro cadastral
    escolas1 = escolas0.drop_duplicates(subset='idinep_orig', keep='first')
    Geocode = Geocoder(escolas1)
    censo_d_escolas = Geocode.Processar()  
    censo_d_escolas = censo_d_escolas.reset_index()
    censo_d_escolas['index'] = censo_d_escolas.index
    censo_d_escolas = censo_d_escolas[['index', 'idinep_orig', 'idcategoria', 'endereco', 'escola', 'idpoint']]
    int_columns = ['index', 'idinep_orig', 'idpoint']
    censo_d_escolas[int_columns] = censo_d_escolas[int_columns].astype(int)

    return censo_d_escolas


def processa_saida (tabela, nome_df, df):
    bcp_file = f"{cd_Bcp}{nome_df}_{ano_final}.csv"
    df.to_csv(bcp_file, sep=';', encoding='utf-8', index=False)
    result = command_bcp(schema='dbo', table=tabela, bcp_file=bcp_file) 
    print("______________________________________________________________")
    print(f"{tabela}: {result}")

## Extract

In [ ]:
if ano_inicial == ano_final:
    base0 = pd.read_csv(f"{cd_CensoEscolar}microdados_ed_basica_{ano_inicial}.csv", encoding='windows-1252', sep=';')
else:
    dfs = []
    for arquivo in os.listdir(cd_CensoEscolar):
        for ano in anos:
            if arquivo.endswith(f"{ano}.csv"):
                df = pd.read_csv(os.path.join(cd_CensoEscolar, arquivo), encoding='windows-1252', sep=';')
                dfs.append(df)
    base0 = pd.concat(dfs, ignore_index=True)

base1, base3 = transform(base0)

### Execute

- **Saída Dimensões**

In [ ]:
print(f"Script iniciado em {inicio}")

censo_d_escolas = trata_dimensoes(base1)
processa_saida('censo_d_escolas-inep', 'censo_d_escolas', censo_d_escolas)

current_datetime = datetime.now()
fim = current_datetime.strftime('%Y-%m-%d %H:%M:%S')

print(f"Script finalizado em {fim}")

- **Saída Fatos**

In [ ]:
print(f"Script iniciado em {inicio}")

censo_colaboradores = gera_fato(base3, 'Colaboradores', 'idarea', ['iddata', 'idarea', 'valor', 'idinep_orig'])
censo_docentes = gera_fato(base3, 'Docentes', 'idcurso', ['iddata', 'idcurso', 'valor', 'idinep_orig'])
censo_matriculas_curso = gera_fato(base3, 'Matrículas', 'idcurso', ['idinep_orig', 'iddata', 'idcurso', 'valor'])
censo_matriculas_curso_integral = gera_fato(base3, 'Matrículas Integral', 'idcurso', ['iddata', 'idcurso', 'valor', 'idinep_orig'])
censo_turmas = gera_fato(base3, 'Turmas', 'idcurso', ['iddata', 'idcurso', 'valor', 'idinep_orig'])


tabelas_dict = {
    'censo_f_matriculas-curso': ('censo_matriculas_curso', censo_matriculas_curso),
    'censo_f_matriculas-integral': ('censo_matriculas_curso_integral', censo_matriculas_curso_integral),
    'censo_f_colaboradores': ('censo_colaboradores', censo_colaboradores),
    'censo_f_docentes': ('censo_docentes', censo_docentes),
    'censo_f_turmas': ('censo_turmas', censo_turmas)
}

for tabela, (nome_df, df) in tabelas_dict.items():
    processa_saida(tabela, nome_df, df)

current_datetime = datetime.now()
fim = current_datetime.strftime('%Y-%m-%d %H:%M:%S')

print(f"Script finalizado em {fim}")